# Creating sub-agent 1: Grafana Assistant
In this folder we will create the first sub-agent the grafana assistant

The grafana assistant gets the alerts at a grafana dashboard as well as the history of alerts.

The agent architecture looks as following:
![Architecture](images/architecture.png)

The agent has an action group
## Prerequisites
Before starting, let's install the required boto3 libraries. Since this is a preview, the required libraries are being installed from the provided `whl` files

**Important:** this command will result in some conflict errors. It is ok to ignore those for now! We will check the pip versions in the next step

In [ ]:
!python3 -m pip install --force-reinstall --no-cache -q -r requirements.txt

## Import required libraries
Next we will import the required libraries. We will also import some support functions available in the parent directory. Those functions are:
- `create_lambda_layer`: to create a lambda layer with the requirements for the agent
- `upload_api_schema`: to upload the API schema file to an s3 bucket
- `create_agent`: helps you to create the necessary IAM permissions and Bedrock agetns based on the agent's name, instructions, foundation models, descriptions and other properties
- `invoke_agent_helper`: helps you to invoke your agent using invoke_agent

You can see the implementation of both functions in the parent directory

In [ ]:
import time
import sys
import os
import configparser

# Get the current file's directory
current_dir = os.path.dirname(os.path.abspath('__file__'))

# Get the parent directory
parent_dir = os.path.dirname(current_dir)
print(parent_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)

from agents import create_lambda_layer, upload_api_schema, create_agent, invoke_agent_helper


#load config
config = configparser.RawConfigParser()
config.read('../devops.properties')

## Create a Lambda layer
Create a lambda layer with the following packages
```
requests
pandas
numpy
```

In [ ]:
agent_name = 'grafana-assistant'
requiments = 'lambda_requirements.txt'
lambda_layer_arn = create_lambda_layer(agent_name, requiments)
lambda_layer_arn

In [ ]:
api_schema_path = 'grafana-agent-open-api.json'
bucket_name, bucket_key = upload_api_schema(agent_name, api_schema_path)
bucket_name, bucket_key

## Defining agent configuration

Let's now define the configuration for our grafana assistant. Let's use the follow instructions to create our agent:

```
You are a cautious and diligent agent. you only with the data that you have accessed from your tools.
You also are able to further filter and select only the data you have got from the api to respond to user requests.
You are a grafana expert and can understand differences between alert, dimension etc.
When a user asks about a particular app's alert status use the dimensionState.

In grafana the name and id correspond to a single alert.
Each alert will have details for individual ap based on different dimensions.
For example MEM alert has alert id 20, but with it , it can track memory for different applications like app1, app2.
You are allowed to use your judgement in mapping the user request to the actual alert name in grafana.

you have access to the following tools or operations
1/ getAlerts : call grafana to get only the latest alert details including the app of the alert, state, alert id and 
last active timestamp. You could optionally pass an App name. When in doubt or when given alert name,
call the api without any additional parameter and use the data subsequently as needed.
2/ getAlertsHistory : Get detailed history of an alert based on the alert id either provided by user or 
fetched using the first tool. you can optionally pass a parameter to get alerts only in the last n relative minutes.
For history of an alert get the alert id from the first tool and call this tool. After getting the data you can filter
it based on request before responding
```

For this agent, we will use `Claude 3 Haiku` model in order to provide faster answers to the user.

In [ ]:
agent_foundation_model = "anthropic.claude-3-haiku-20240307-v1:0"
agent_instruction = """You are a cautious and diligent agent. you only with the data that you have accessed from your tools.
You also are able to further filter and select only the data you have got from the api to respond to user requests.
You are a grafana expert and can understand differences between alert, dimension etc.
When a user asks about a particular app's alert status use the dimensionState.

In grafana the name and id correspond to a single alert.
Each alert will have details for individual ap based on different dimensions.
For example MEM alert has alert id 20, but with it , it can track memory for different applications like app1, app2.
You are allowed to use your judgement in mapping the user request to the actual alert name in grafana.

you have access to the following tools or operations
1/ getAlerts : call grafana to get only the latest alert details including the app of the alert, state, alert id and 
last active timestamp. You could optionally pass an App name. When in doubt or when given alert name,
call the api without any additional parameter and use the data subsequently as needed.
2/ getAlertsHistory : Get detailed history of an alert based on the alert id either provided by user or 
fetched using the first tool. you can optionally pass a parameter to get alerts only in the last n relative minutes.
For history of an alert get the alert id from the first tool and call this tool. After getting the data you can filter
it based on request before responding"""
agent_description = "Agent to manage grafana actions"


action_group_config = {
    'name': 'GrafanaActionGroup',
    'description': 'Action group to manage grafana dashboard actions for getting alerts and alerts history',
    'lambda_function_name': f'{agent_name}-lambda',
    'lambda_file_path': 'lambda_function.py',
    'api_schema': {
        'bucket_name': bucket_name,
        'bucket_key': bucket_key
    },
    'lambda_layers': [lambda_layer_arn],
    'environment': {
        'Variables': {
            'grafana_url': config.get('grafana', 'grafana.url'), # grafana url including the https e.g. https://g-bb69dfe7ae.grafana-workspace.us-east-1.amazonaws.com
            'grafana_token': config.get('grafana', 'grafana.token')
        }
    }
}

## Create agent
Next we will create the agent with the provided information

In [ ]:
grafana_agent_id, grafana_assistant_agent_alias_id, grafana_assistant_agent_alias_arn = create_agent(
    agent_name,
    agent_instruction,
    agent_foundation_model=agent_foundation_model,
    agent_description=agent_description,
    action_group_config=action_group_config
)
time.sleep(20)

## Getting details from the agent

Let's take a look at the details from the created agent. We will need its `grafana_assistant_agent_alias_arn` to associate the agent with its supervisor agent

In [ ]:
grafana_agent_id, grafana_assistant_agent_alias_id, grafana_assistant_agent_alias_arn

## Testing Agent
Now that we've created the agent, let's test it by using our `invoke_agent_helper` function

### Getting started
First let's get the history of alerts for app1

In [ ]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = """can you get me alert history of memory alert for the app app1"""
response = invoke_agent_helper(
    query, session_id, grafana_agent_id, grafana_assistant_agent_alias_id, enable_trace=True
)
print(response)

In [ ]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = """when was the last Normal state for the response time for app4"""
response = invoke_agent_helper(
    query, session_id, grafana_agent_id, grafana_assistant_agent_alias_id, enable_trace=True
)
print(response)

In [ ]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = """what all alerts are currently firing"""
response = invoke_agent_helper(
    query, session_id, grafana_agent_id, grafana_assistant_agent_alias_id, enable_trace=True
)
print(response)

## Saving the information
Let's now save some information so that we can use this agent as a sub-agent of our traveler assistant

In [ ]:
%store grafana_agent_id
%store grafana_assistant_agent_alias_id
%store grafana_assistant_agent_alias_arn

## Next steps
Next let's create our other sub-agents for github